In [1]:
import torch
from torch import nn
from torchvision.utils import make_grid, save_image

def create_noise(sample_size, nz):
    return torch.randn(sample_size, nz).to('cpu')

# to save the images generated by the generator
def save_generator_image(image, path):
    save_image(image, path)

class Generator(nn.Module):
    def __init__(self, nz):
        super(Generator, self).__init__()
        self.nz = nz
        self.main = nn.Sequential(
            nn.Linear(self.nz + 10, 256),
            nn.ReLU(),

            nn.Linear(256, 512),
            nn.ReLU(),

            nn.Linear(512, 784),
            nn.Tanh(),
        )

    def forward(self, x, k):
        x = torch.cat((x, k), dim=1)
        # print(x)
        return self.main(x).view(-1, 1, 28, 28)


In [2]:
import numpy as np

nz = 128
n=10

generator = Generator(nz=nz)

generator.load_state_dict(torch.load("./models/conditional/generator.pth", map_location=torch.device('cpu')))


<All keys matched successfully>

In [9]:

onehot = torch.zeros((n*3, n))

print(onehot)
onehot[np.arange(n*3), np.array([[i,i,i] for i in range(n)]).flatten()] = 1
print(onehot)


tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],


In [13]:

noise = create_noise(n*3, nz)

generated_img = generator(noise, onehot).cpu().detach()
# make the images as grid
generated_img = make_grid(generated_img, nrow = 3)
save_generator_image(generated_img, f"./models/conditional/conditional_generated_{n}.png")

In [4]:
import os
import random
from PIL import Image, ImageOps


def concat_images(image_paths, size, shape=None):
    # Open images and resize them
    width, height = size
    images = map(Image.open, image_paths)
    images = [ImageOps.fit(image, size, Image.ANTIALIAS) 
              for image in images]
    
    # Create canvas for the final image with total size
    shape = shape if shape else (1, len(images))
    image_size = (width * shape[1], height * shape[0])
    image = Image.new('RGB', image_size)
    
    # Paste images into final image
    for row in range(shape[0]):
        for col in range(shape[1]):
            offset = width * col, height * row
            idx = row * shape[1] + col
            image.paste(images[idx], offset)
    
    return image

# Get list of image paths
folder = './models/wasserstein'
image_array = [os.path.join(folder, f) 
               for f in os.listdir(folder) if f.endswith('1.png')]

# Random selection of images
# image_array = random.choices(image_paths, k=8)

# Create and save image grid
image = concat_images(image_array, (300, 300), (2, 2))
image.save('image.jpg', 'JPEG')